In [1]:
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats

In [2]:
# Import data file
data_directory = 'C:/Users/mvmigem/Documents/data/project_1/raw_data/'
dir_list = glob.glob(data_directory+'sub_*/behav/*.csv')


In [3]:
df_list = []
for i in range(len(dir_list)):
    behav_dat = pd.read_csv(dir_list[i])
    df_list.append(behav_dat)

big_df = pd.concat(df_list,ignore_index=True)

In [15]:
"""
Counting the amount of raw events
"""
sub_list = big_df['participant'].unique() 
# Step 1: Group by subject, attention, and expectation, and count rows
df_repeated = big_df.loc[np.repeat(big_df.index, 4)].reset_index(drop=True)
df_to_count = df_repeated[~(df_repeated.participant.isin([3,7,20]))]
grouped = df_to_count.groupby(['participant', 'attention', 'expected']).size().reset_index(name='row_count')

# Step 3: Group by attention and expectation, and calculate the mean and standard deviation
stats_per_condition = grouped.groupby(['attention', 'expected'])['row_count'].agg(['mean', 'std']).reset_index()


In [4]:
def calculate_subtraction(row):
    if row['catch_trial'] != 0:
        relevant_timepoint_col = f"t_stim_{row['catch_trial']}"
        rt =  row['press_time'] - row[relevant_timepoint_col]
        if rt > 3200:
            return np.nan
        return rt
# Apply the function to create the subtraction column
big_df['rt'] = big_df.apply(calculate_subtraction, axis=1)

In [ ]:
"""
***Aestetics***
"""
# load color palette
my_palette = sns.color_palette().as_hex()
sns.color_palette().as_hex()


In [ ]:
subject_loc_quad = big_df.groupby('participant')['loc_quad'].first()

ax = sns.histplot( x=subject_loc_quad,bins = 4, palette=my_palette[:4], discrete=True)

tick_labels = ['top left', 'top right', 'bottom right','bottom left']  # Replace ... with additional labels

ax.set_xticks(range(4),tick_labels)
ax.set_xlabel('located quadrant')
plt.show()

In [ ]:
palette=my_palette[:4]

In [ ]:
ax = sns.barplot(data= big_df, x='attention',y='rt',
                 edgecolor= 'black', palette= [my_palette[4], my_palette[5]],
                errorbar='se', errwidth= 1.5,
                capsize=.1,width=.6, linewidth=0.7)


In [8]:
sub_grouped = big_df.groupby(['participant','attention'])['rt'].mean()
sub_grouped = sub_grouped.unstack()
t_statistic, p_value = stats.ttest_rel(sub_grouped['attended'], sub_grouped['unattended'])

In [ ]:
ax = sns.barplot(data= sub_grouped,
                 edgecolor= 'black', palette= [my_palette[4], my_palette[5]],
                errorbar='se', errwidth= 1.5,
                capsize=.1,width=.6, linewidth=0.7)

In [ ]:
excuded_pp = [3,14,20]
pp_info = big_df.groupby('participant').first().reset_index()
pp_info = pp_info[~pp_info['participant'].isin(excuded_pp)]



In [ ]:
pp_info['handed'].value_counts()